In [ ]:
# For fitting peaks. To be used from the interactive notebook

In [ ]:
# Generate model creator string
creator_string = "Model(uniform, Parameter(value=0, bounds=(-50, 50), label='background'), label='background'),"
peaks = np.concatenate((peaks1,peaks2,peaks3)).astype(int)
peaks = np.sort(peaks)
for counter, i in enumerate(peaks):
    creator_string += "Model(gaussian1D, Parameter(value={amplitude}, bounds=(0, 100),  label='amplitude'), Parameter(value={location}, bounds=(min(k)-.01, max(k)+.01), label='center'),Parameter(value=.0012,  bounds=(.0001, .01), label='width'), label='peak_{index}'),".format(location=k[i], amplitude=a[i], index=counter)
creator_string += "label='gaussian_peaks')"
creator_string = "model = CompositeModel(" + creator_string

In [ ]:
from dataphile.datasets import SyntheticDataset
from dataphile.statistics.distributions import gaussian1D, uniform
from dataphile.statistics.regression.modeling import Parameter, Model, CompositeModel, AutoGUI
from matplotlib import patches
#plt.style.use('seaborn-notebook')
%matplotlib notebook

figure = plt.figure('Absorption Data', figsize=(18, 9))

# create main plot of data
ax_1 = figure.add_axes([0.02, .64, .96, 0.33])
data_graph, = ax_1.step(k, a_nosmooth, color='steelblue', alpha=.6, lw=1, label='data')

# labels
ax_1.set_ylabel('Absorption', labelpad=15, fontweight='semibold')
ax_1.set_xlabel('Wavenumber (cm^-1)', labelpad=15, fontweight='semibold')

# overlay small markers showing location of features
for counter, i in enumerate(peaks):
    ax_1.scatter(k[i], a[i]+10, marker='v', color='steelblue')
    ax_1.text(k[i], a[i]+13, '{}'.format(counter), horizontalalignment='center')
    
# xloc, yloc = x0_s, [(bias + peak).max() + 25 for peak in peaks]
# markers = ax_1.scatter(xloc, yloc, marker='v', color='steelblue')


# overlay small markers showing location of features
#loc, yloc = x0_s, [(bias + peak).max() + 25 for peak in peaks]
#markers = ax_2.scatter(xloc, yloc, marker='v', color='steelblue')

exec(creator_string)

xsample = np.linspace(min(k), max(k), 5000)
model_graph, = ax_1.plot(xsample, model(xsample), color='black', label='model')
ax_1.legend();

gui = AutoGUI(model, [model_graph], bbox=[.02, 0.06, .96, .6], figure=figure,
              slider_options={'color': 'steelblue'}, data=(k, a_nosmooth));

In [ ]:
# Text boxes for more fine control, linked to sliders
amp_text = widgets.Text(
    value='',
    placeholder='e.g. 42',
    description='Adjust amplitude:',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='30%'),
    disabled=False
)
cen_text = widgets.Text(
    value='',
    placeholder='e.g. 17343.5',
    description='Adjust center:',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='30%'),
    disabled=False
)
sig_text = widgets.Text(
    value='',
    placeholder='e.g. .001',
    description='Adjust width:',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='30%'),
    disabled=False
)
adjust_button = widgets.Button(
    description='Adjust',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    icon='comic-sans'
)
adjust_button.style.button_color = 'steelblue'
adjust_button.style.font_weight = 'bold'

def adjust_button_clicked(b):
    if len(gui.sliders) == 3:
        if amp_text.value != '': gui.sliders[0].value = float(amp_text.value)
        if cen_text.value != '': gui.sliders[1].value = float(cen_text.value)
        if sig_text.value != '': gui.sliders[2].value = float(sig_text.value)
        

display(widgets.HBox([amp_text, cen_text, sig_text, adjust_button], layout=widgets.Layout(width='900px')))
adjust_button.on_click(adjust_button_clicked)